In [ ]:
import itertools

import numpy as np
import pandas as pd
import tensorly as tl
from tensorly.cp_tensor import cp_to_tensor
import matplotlib.pyplot as plt
import random
np.random.seed(42)

In [ ]:
# Parameters

alpha = [100, 1000, 10000, 100000] # Varying parameter: Influence parameter
lambda_ = [1000, 10000] # Varying parameter: Drift parameter
sigma = [50, 80] # Varying parameter: Diffusion parameter
dt = [0.01] # Varying parameter: Time step
n_particles = [100, 150] # Varying parameter: Number of particles, number of initial random tensors
T = 1.0 # Fixed parameter: Total time
#n_iterations = int(T / dt) # Varying parameter: Number of iterations

# Create itertools.product to generate all combinations of parameters
param_grid = list(itertools.product(dt, n_particles, alpha, lambda_, sigma))

I, J, K = 3, 3, 3 # Fixed parameters: Dims of initial tensor
rank = 2 # Fixed parameter: number of rank-1 tensors

# Create a random tensor (change np.random.seed(--) in case of need)
tensor = tl.tensor(np.random.random((I, J, K)))

In [ ]:
tensor

In [ ]:
# particles[i], where i = [A_i, B_i, C_i] and A_i.shape = 10x5, B_i.shape = 12x5, C_i.shape = 14x5
# a{r}_{i} = particles[i-1]['A'][:,r-1]
# where r = rank from 1 to 5, i = particles from 1 to n_particles

"""
# example indexing:
a1_1 = particles[0]['A'][:,0]
b1_1 = particles[0]['B'][:,0]
c1_1 = particles[0]['C'][:,0]

a2_1 = particles[0]['A'][:,1]
b2_1 = particles[0]['B'][:,1]
c2_1 = particles[0]['C'][:,1]

a3_1 = particles[0]['A'][:,2]
b3_1 = particles[0]['B'][:,2]
c3_1 = particles[0]['C'][:,2]

a4_1 = particles[0]['A'][:,3]
b4_1 = particles[0]['B'][:,3]
c4_1 = particles[0]['C'][:,3]

a5_1 = particles[0]['A'][:,4]
b5_1 = particles[0]['B'][:,4]
c5_1 = particles[0]['C'][:,4]

# ----------------------------

a1_2 = particles[1]['A'][:,0]
b1_2 = particles[1]['B'][:,0]

# and so on"""

In [ ]:
def objective_function(particle):
    # Reconstruct tensor from the given particle
    A = particle['A']
    B = particle['B']
    C = particle['C']
    reconstructed_tensor = cp_to_tensor((np.ones(rank), [A, B, C]))

    # Compute the Frobenius norm of the difference
    error = tl.norm(tensor - reconstructed_tensor)

    return error

In [ ]:
def compute_consensus_point(particles, alpha):
    # Find the particle with the minimum energy (V* in the image)
    min_energy_particle = min(particles, key=objective_function)
    min_energy = objective_function(min_energy_particle)

    # Initialize the numerators and the denominator
    numerator_A = np.zeros_like(particles[0]['A'])
    numerator_B = np.zeros_like(particles[0]['B'])
    numerator_C = np.zeros_like(particles[0]['C'])
    denominator = 0.0

    for particle in particles:
        A, B, C = particle['A'], particle['B'], particle['C']

        # Compute the energy for the current particle
        energy = objective_function(particle)

        # Apply the numerical stabilization trick
        weight = np.exp(-alpha * (energy - min_energy))

        # Update the numerators and the denominator
        numerator_A += weight * A
        numerator_B += weight * B
        numerator_C += weight * C
        denominator += weight

    # Compute the consensus matrices (now no need for epsilon)
    consensus_A = numerator_A / denominator
    consensus_B = numerator_B / denominator
    consensus_C = numerator_C / denominator

    return {'A': consensus_A, 'B': consensus_B, 'C': consensus_C}


In [ ]:
"""def compute_consensus_point(particles, alpha):
    numerator_A = np.zeros_like(particles[0]['A'])
    numerator_B = np.zeros_like(particles[0]['B'])
    numerator_C = np.zeros_like(particles[0]['C'])
    denominator = 0.0

    for particle in particles:
        A, B, C = particle['A'], particle['B'], particle['C']

        # Compute the energy for the current particle (Frobenius norm error)
        energy = objective_function(particle)

        # Compute the weight for this particle
        weight = np.exp(-alpha * energy)

        # Update the numerators and the denominator
        numerator_A += weight * A
        numerator_B += weight * B
        numerator_C += weight * C
        denominator += weight

    # Add a small epsilon to avoid division by zero
    epsilon = 1e-10
    denominator = np.maximum(denominator, epsilon)

    # Compute the consensus matrices
    consensus_A = numerator_A / denominator
    consensus_B = numerator_B / denominator
    consensus_C = numerator_C / denominator

    return {'A': consensus_A, 'B': consensus_B, 'C': consensus_C}"""

In [ ]:
def projection_operator_matrix(V):
    """Apply the projection operator P(V) = I - (V ⊗ V) / |V|^2 to each column of the matrix V."""
    V_norm_sq = np.sum(V**2, axis=0, keepdims=True)  # Compute |V|^2 for each column (1 x n_columns)
    outer_product = np.einsum('ik,jk->ijk', V, V)  # Compute V ⊗ V for each column (n_rows x n_rows x n_columns)
    I = np.eye(V.shape[0])  # Identity matrix of appropriate size (n_rows x n_rows)
    P = I[:, :, np.newaxis] - outer_product / V_norm_sq  # Apply the projection operator column-wise
    return P

In [ ]:
def isotropic_update(particles, consensus_point, lambda_, sigma, dt):
    for particle in particles:
        """
        Perform isotropic updates on all particles.

        Parameters:
        - particles: list of particle dictionaries with keys 'A', 'B', 'C'.
        - consensus_point: dictionary with keys 'A', 'B', 'C' representing the consensus matrices.
        - lambda_: drift parameter.
        - sigma: diffusion parameter.
        - dt: time step.
        """

        # Extract the matrices A, B, C from the particle and the consensus
        A, B, C = particle['A'], particle['B'], particle['C']
        A_consensus, B_consensus, C_consensus = consensus_point['A'], consensus_point['B'], consensus_point['C']

        if objective_function(compute_consensus_point(particles, alpha)) < objective_function(particle):
            drift_A = (-lambda_) * (A - A_consensus) * dt
            drift_B = (-lambda_) * (B - B_consensus) * dt
            drift_C = (-lambda_) * (C - C_consensus) * dt
        else:
            drift_A = np.zeros(A.shape).tolist()
            drift_B = np.zeros(B.shape).tolist()
            drift_C = np.zeros(C.shape).tolist()

        # Isotropic noise term, applied column_wise
        noise_A = sigma * np.abs(A - A_consensus) * np.random.randn(*A.shape) * dt
        noise_B = sigma * np.abs(B - B_consensus) * np.random.randn(*B.shape) * dt
        noise_C = sigma * np.abs(C - C_consensus) * np.random.randn(*C.shape) * dt

        # Update particle's A, B, and C matrices
        particle['A'] += np.array(drift_A) + np.array(noise_A)
        particle['B'] += np.array(drift_B) + np.array(noise_B)
        particle['C'] += np.array(drift_C) + np.array(noise_C)


In [ ]:
def anisotropic_update(particles, consensus_point, lambda_, sigma, dt):
    for particle in particles:
        # Extract the matrices A, B, C from the particle and the consensus
        A, B, C = particle['A'], particle['B'], particle['C']
        A_consensus, B_consensus, C_consensus = consensus_point['A'], consensus_point['B'], consensus_point['C']

        # Compute the elementwise differences
        diff_A = A - A_consensus
        diff_B = B - B_consensus
        diff_C = C - C_consensus

        # Compute the projection terms for each matrix
        P_A = projection_operator_matrix(A)
        P_B = projection_operator_matrix(B)
        P_C = projection_operator_matrix(C)

        # Generate Brownian motion term
        delta_B_A = np.random.randn(*A.shape) * np.sqrt(dt)
        delta_B_B = np.random.randn(*B.shape) * np.sqrt(dt)
        delta_B_C = np.random.randn(*C.shape) * np.sqrt(dt)

        # Corrected drift term: includes projection P(V_ti) applied column-wis
        drift_A = lambda_ * dt * np.einsum('ijk,jk->ik', P_A, A_consensus)
        drift_B = lambda_ * dt * np.einsum('ijk,jk->ik', P_B, B_consensus)
        drift_C = lambda_ * dt * np.einsum('ijk,jk->ik', P_C, C_consensus)

        # Anisotropic noise term, projected and applied column-wise
        D_A = np.diag(np.linalg.norm(diff_A, axis=0)**2)
        noise_A = sigma * np.matmul(diff_A, D_A) * delta_B_A

        D_B = np.diag(np.linalg.norm(diff_B, axis=0)**2)
        noise_B = sigma * np.matmul(diff_B, D_B) * delta_B_B

        D_C = np.diag(np.linalg.norm(diff_C, axis=0)**2)
        noise_C = sigma * np.matmul(diff_C, D_C) * delta_B_C

        # Additional correction term applied column-wise
        correction_A = -0.5 * dt * sigma**2 * (np.linalg.norm(diff_A, axis=0, keepdims=True)**2 * A)
        correction_B = -0.5 * dt * sigma**2 * (np.linalg.norm(diff_B, axis=0, keepdims=True)**2 * B)
        correction_C = -0.5 * dt * sigma**2 * (np.linalg.norm(diff_C, axis=0, keepdims=True)**2 * C)

        # Update particle's A, B, and C matrices
        particle['A'] += drift_A + noise_A + correction_A
        particle['B'] += drift_B + noise_B + correction_B
        particle['C'] += drift_C + noise_C + correction_C

In [ ]:
"""# List to store reconstruction errors of the consensus point at each iteration
consensus_errors = []

# Main iteration loop
for iteration in range(n_iterations):
    # Step 1: Calculate the consensus point
    consensus_point = compute_consensus_point(particles, alpha=alpha)

    # Step 2: Compute the reconstruction error for the consensus point
    consensus_tensor = cp_to_tensor((np.ones(rank), [consensus_point['A'], consensus_point['B'], consensus_point['C']]))
    consensus_error = tl.norm(tensor - consensus_tensor)

    # Print the reconstruction error of the consensus point
    print(f"Iteration {iteration + 1}/{n_iterations}, Consensus Reconstruction Error: {consensus_error}")

    # Store the error for later plotting
    consensus_errors.append(consensus_error)

    # Step 3: Update the particles
    isotropic_update(particles, consensus_point, lambda_, sigma, dt)
    # or use anisotropic_update(particles, consensus_point, lambda_, sigma, dt) if desired

# Final output: You could also save or analyze the final consensus point
final_consensus_point = compute_consensus_point(particles, alpha=alpha)
final_consensus_tensor = cp_to_tensor((np.ones(rank), [final_consensus_point['A'], final_consensus_point['B'], final_consensus_point['C']]))
final_error = tl.norm(tensor - final_consensus_tensor)

print("Final Consensus Reconstruction Error: ", final_error)

# Plotting the consensus reconstruction errors over iterations
plt.plot(consensus_errors, label='Consensus Reconstruction Error')
plt.xlabel('Iteration')
plt.ylabel('Reconstruction Error')
plt.title('Reconstruction Error of Consensus Point Over Iterations')
plt.legend()
plt.grid(True)
plt.show()"""

In [ ]:
results = []
# Create itertools.product to generate all combinations of parameters
param_grid = list(itertools.product(dt, n_particles, alpha, lambda_, sigma))

# Iterate over the parameter grid
for params in param_grid:
    dt, n_particles, alpha, lambda_, sigma = params

    n_iterations = int(T / dt) # Varying parameter: Number of iterations

    # Step 2: Initialize
    particles = []
    for _ in range(n_particles):
        A = np.random.randn(I, rank) # n_particles Ixrank matrices
        B = np.random.randn(J, rank) # n_particles Jxrank matrices
        C = np.random.randn(K, rank) # n_particles Kxrank matrices
        particles.append({'A': A, 'B': B, 'C': C})

    # Print all parameters including fixed ones
    print(f"Running grid search with params: dt={dt}, n_particles={n_particles}, alpha={alpha}, lambda_={lambda_}, sigma={sigma}, T={T}, I={I}, J={J}, K={K}, rank={rank}, #iters={n_iterations}")

    # List to store reconstruction errors of the consensus point at each iteration
    consensus_errors = []

    # Main iteration loop
    for iteration in range(n_iterations):
        # Step 1: Calculate the consensus point
        consensus_point = compute_consensus_point(particles, alpha=alpha)

        # Step 2: Compute the reconstruction error for the consensus point
        consensus_tensor = cp_to_tensor((np.ones(rank), [consensus_point['A'], consensus_point['B'], consensus_point['C']]))
        consensus_error = tl.norm(tensor - consensus_tensor)

        # Print the reconstruction error of the consensus point
        print(f"Iteration {iteration + 1}/{n_iterations}, Consensus Reconstruction Error: {consensus_error}")

        # Store the error for later plotting
        consensus_errors.append(consensus_error)

        # Step 3: Update the particles (either isotropic or anisotropic)
        isotropic_update(particles, consensus_point, lambda_, sigma, dt)

    # Collect errors at specific iterations: 1st, 1/4th, 1/2th, 3/4th, and last iteration
    error_begin = consensus_errors[0]
    error_iter_1_4 = consensus_errors[int(n_iterations * 0.25)]
    error_iter_1_2 = consensus_errors[int(n_iterations * 0.5)]
    error_iter_3_4 = consensus_errors[int(n_iterations * 0.75)]
    error_last = consensus_errors[-1]

    # Add the parameters and errors to the results list
    results.append({
        'alpha': alpha,
        'lambda': lambda_,
        'sigma': sigma,
        'T': T,
        'dt': dt,
        '#iters (T/dt)': n_iterations,
        '#particles': n_particles,
        'I': I,
        'J': J,
        'K': K,
        'rank': rank,
        'error_begin': error_begin,
        'error_iter 1/4': error_iter_1_4,
        'error_iter 1/2': error_iter_1_2,
        'error_iter 3/4': error_iter_3_4,
        'error_iter last': error_last,
        'landscape': ''  # Fill this field if needed
    })

    # Final output: Save or analyze the final consensus point
    final_consensus_point = compute_consensus_point(particles, alpha=alpha)
    final_consensus_tensor = cp_to_tensor((np.ones(rank), [final_consensus_point['A'], final_consensus_point['B'], final_consensus_point['C']]))
    final_error = tl.norm(tensor - final_consensus_tensor)

    print("Final Consensus Reconstruction Error: ", final_error)

    # Plotting the consensus reconstruction errors over iterations
    plt.plot(consensus_errors, label=f'Consensus Error (dt={dt}, np={n_particles}, α={alpha}, λ={lambda_}, σ={sigma})')

# Final plot formatting (if multiple runs are plotted)
plt.xlabel('Iteration')
plt.ylabel('Reconstruction Error')
plt.title('Reconstruction Error of Consensus Point Over Iterations (Grid Search)')
plt.legend()
plt.grid(True)
plt.show()

# Convert results to a pandas DataFrame
df_results = pd.DataFrame(results)

Iteration 19/100, Consensus Reconstruction Error: 2.9089886144862174
Iteration 20/100, Consensus Reconstruction Error: 2.9089886144862174
Iteration 21/100, Consensus Reconstruction Error: 2.9089886144862174
Iteration 22/100, Consensus Reconstruction Error: 2.9089886144862174
Iteration 23/100, Consensus Reconstruction Error: 2.9089886144862174
Iteration 24/100, Consensus Reconstruction Error: 2.9089886144862174
Iteration 25/100, Consensus Reconstruction Error: 2.9089886144862174
Iteration 26/100, Consensus Reconstruction Error: 2.9089886144862174
Iteration 27/100, Consensus Reconstruction Error: 2.9089886144862174
Iteration 28/100, Consensus Reconstruction Error: 2.9089886144862174
Iteration 29/100, Consensus Reconstruction Error: 2.9089886144862174
Iteration 30/100, Consensus Reconstruction Error: 2.9089886144862174


KeyboardInterrupt: 

In [ ]:
df_results